In [1]:
import soothsayer as sy

# %run ~/Google/.ipython-profile.py
def tqdm(*args, **kwargs):
    if hasattr(tqdm_base, '_instances'):
        for instance in list(tqdm_base._instances):
            tqdm_base._decr_instances(instance)
    return tqdm_base(*args, **kwargs)

Soothsayer Ecosystem
 * Soothsayer v2021.07.27
 * Soothsayer Utilities v2021.07.27
 * Compositional v2020.12.16
 * Hive NetworkX v2021.05.18
 * Ensemble NetworkX v2021.06.24


In [4]:
# df_exons = sy.io.read_gtf_gff_base("/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.105.gtf.gz", compression="gzip", record_type="exon", verbose=True)

Processing lines: 100%|██████████| 1866443/1866443 [00:04<00:00, 425884.46it/s]


In [43]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import gzip
from tqdm import tqdm

path_assembly = "/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz"
path_gtf = "/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.105.gtf.gz"


# Assembly
reference_to_sequence = OrderedDict()
with {True:gzip.open, False:open}[path_assembly.endswith(".gz")](path_assembly, "rt") as f:
    for id_record, sequence in tqdm(SimpleFastaParser(f), desc="Reading: {}".format(path_assembly), unit=" sequence"):
        id_record = id_record.split(" ")[0]
        reference_to_sequence[id_record] = sequence
reference_to_sequence = pd.Series(reference_to_sequence)



Reading: /Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz: 61 sequence [00:33,  1.82 sequence/s]


In [81]:
# GTF
exon_info = list()
with {True:gzip.open, False:open}[path_gtf.endswith(".gz")](path_gtf, "rt") as f:
    for line in tqdm(f.readlines(), desc="Reading: {}".format(path_gtf), unit=" line"):
        line = line.strip()
        if "\texon\t" in line:
            fields = line.split("\t")
            id_reference = fields[0]
            start = int(fields[3])
            end = int(fields[4])
            sense = fields[6]
            id_gene = fields[-1].split('gene_id "')[1].split('"')[0] # Need a regex instead
            id_transcript = fields[-1].split('transcript_id "')[1].split('"')[0] # Need a regex instead
            exon_number = fields[-1].split('exon_number "')[1].split('"')[0] # Need a regex instead
            transcript_version = fields[-1].split('transcript_version "')[1].split('"')[0] # Need a regex instead
            exon_info.append([id_reference, start, end, sense, exon_number, id_gene, "{}.{}".format(id_transcript,transcript_version)])
        
df_exons = pd.DataFrame(exon_info, columns=["reference_id", "start", "end", "sense", "exon_number", "gene_id", "transcript_id"]).sort_values(["reference_id","start"]).reset_index(drop=True)


Reading: /Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.105.gtf.gz: 100%|██████████| 1866443/1866443 [00:10<00:00, 177434.69 line/s]


In [75]:
# + ENSMUST00000161581
# - ENSMUST00000001008
# i = 0 
# for id_transcript, df in tqdm(df_exons.query("sense == '-'").groupby("transcript_id", axis=0), desc="Merging exon sequences", unit=" transcript"):
#     if df.shape[0] == 4:
#         i += 1
#         if i == 4:
#             break
# df

df_exons.query("transcript_id == 'ENSMUST00000000001'")

,reference_id,start,end,sense,exon_number,gene_id,transcript_id
491009,3,108014596,108016632,-,9,ENSMUSG00000000001,ENSMUST00000000001
491010,3,108016719,108016928,-,8,ENSMUSG00000000001,ENSMUST00000000001
491011,3,108019251,108019404,-,7,ENSMUSG00000000001,ENSMUST00000000001
491012,3,108019789,108019918,-,6,ENSMUSG00000000001,ENSMUST00000000001
491013,3,108023079,108023207,-,5,ENSMUSG00000000001,ENSMUST00000000001
491014,3,108025617,108025774,-,4,ENSMUSG00000000001,ENSMUST00000000001
491015,3,108030858,108030999,-,3,ENSMUSG00000000001,ENSMUST00000000001
491016,3,108031111,108031153,-,2,ENSMUSG00000000001,ENSMUST00000000001
491017,3,108053204,108053462,-,1,ENSMUSG00000000001,ENSMUST00000000001


In [82]:
complement = str.maketrans('ACGTacgt','TGCAtgca')

def reverse_complement(sequence):
    """
    Inputs a sequence of DNA and returns the reverse complement se$
    Outputs a reversed string of the input where A>T, T>A, G>C, an$
    """

    reverse_complement = sequence.translate(complement)[::-1]
    return reverse_complement


transcript_to_premRNA = dict()

error_transcripts = list()
for id_transcript, df in tqdm(df_exons.groupby("transcript_id", axis=0), desc="Merging exon sequences", unit=" transcript"):
    values = df.values
    number_of_exons = values.shape[0]
    id_reference = str(values[0][0])
    sense = values[0][3]
    
    exon_sequences = list()
    if id_reference in reference_to_sequence:
        sequence_reference = reference_to_sequence[id_reference]
        
        # More than 1 exon
        if number_of_exons > 1:
            for i in range(0, number_of_exons):
                row = values[i]
                exon_start = row[1] - 1 
                exon_end = row[2]
                sequence_exon = sequence_reference[exon_start:exon_end].upper()
                exon_sequences.append(sequence_exon)
                
                if i < (number_of_exons - 1):
                    intron_start = exon_end
                    intron_end = values[i+1][1] - 1
                    sequence_intron = sequence_reference[intron_start:intron_end].lower()
                    exon_sequences.append(sequence_intron)
                    
        # Only 1 exon
        else:
            exon_start = values[0][1] - 1
            exon_end = values[0][2]
            sequence_exon = sequence_reference[exon_start:exon_end]
            exon_sequences.append(sequence_exon)
            
        sequence_premRNA = "".join(exon_sequences)
        
        if sense == "-":
            sequence_premRNA = reverse_complement(sequence_premRNA)
            
        transcript_to_premRNA[id_transcript] = sequence_premRNA
    else:
        error_transcripts.append(values)
transcript_to_premRNA = pd.Series(transcript_to_premRNA)


Merging exon sequences: 100%|██████████| 142435/142435 [00:38<00:00, 3653.74 transcript/s]


In [84]:
# + ENSMUST00000161581
# - ENSMUST00000000001
# transcript_to_premRNA["ENSMUST00000000001"]
transcript_to_premRNA

ENSMUST00000000001.5     CACACATCCGGTTCTTCCGGGAGCTAGGGGAGCTGACGGAGAAGGC...
ENSMUST00000000003.14    GTCAGTGCACAACTGCCAACTGGGATGCAGAACACTGCTCACGCCA...
ENSMUST00000000010.9     GGTCCGTGTGCCACCTTTTCCCTGCTTGGGCGCCGCGGCGCGAGCG...
ENSMUST00000000028.14    TGGAAACACATTCAAATAATGTGTGACTGAATTTACTTTATGTCTA...
ENSMUST00000000033.12    GGCACTGACCAGTTCGCAAACTGGACATTAGCTTCTCCTGTGAGAA...
                                               ...                        
ENSMUST00020183809.1     AGGTGAATGGAGTAACCTGACAGCGGGAACGAGACGACGGCGAGCG...
ENSMUST00020183810.1     GTCATCGCCACCATCCCTACCAGCAGGCTCAAGTTCCTGAAAGAGG...
ENSMUST00020183811.1     GGCTGCGCGGGGAATTCGAATCGCCGGCGGCCTTCGAGACTCGGGG...
ENSMUST00020183812.1     GTTGCGTTACTTCCGCTGGTGCAGGAGCTGAGTTTCCGCGGGATCA...
ENSMUST00020183813.1     AAGCTGTCTCTCCTGAGCTCTTTCATGTTATTGATGACTTTGTGAA...
Length: 142435, dtype: object

In [86]:
sequences_cds = sy.io.read_fasta("/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.cds.all.fa.gz", False)
sequences_cds

ENSMUST00000178537.2                                          GGGACAGGGGGC
ENSMUST00000178862.2                                        GGGACTGGGGGGGC
ENSMUST00000196221.2                                             ATGGCATAT
ENSMUST00000177564.2                                      ATCGGAGGGATACGAG
ENSMUST00000179520.2                                           CTAACTGGGAC
                                               ...                        
ENSMUST00000052927.11    ATGGCCAGCCACGTGGACCTGCTCACAGAGCTGCAGCTGCTGGAGA...
ENSMUST00000103116.10    ATGGCCAGCCACGTGGACCTGCTCACAGAGCTGCAGCTGCTGGAGA...
ENSMUST00000045503.11    ATGGCCAGCCACGTGGACCTGCTCACAGAGCTGCAGCTGCTGGAGA...
ENSMUST00000145073.2     GGACGTCGCAAGAAGGTGTCCTTCGAGGCCAGTGTAGCGTTGCTGG...
ENSMUST00000099133.11    ATGACAGAGCCGTCCCTTCTCCCTGCTGCCAAGATGGAAGGGCTGG...
Name: /Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.cds.all.fa.gz, Length: 67165, dtype: object

In [88]:
transcript_to_premRNA["ENSMUST00000178537.2"]

'GGGACAGGGGGC'

In [ ]:
fields = ['seq_record', 'pos_start', 'pos_end', 'sense', 'transcript_id',  'exon_id']
df_data = df_exons.loc[:,fields].sort_values(["seq_record", "pos_start"], ascending=True)

for id_field in ['pos_start', 'pos_end']:
    df_exons[id_field] = df_data[id_field].astype(int)
d_id_premrna = OrderedDict()
not_work = list()
for id_transcript, _df_exon in tqdm(df_data.groupby("transcript_id", axis=0)):
#     id_transcript = id_transcript[11:]
#     print(id_transcript)
    data = _df_exon.values
    num_exons = data.shape[0]
    id_reference = str(data[0][0])
    sense = data[0][3]
    seq_transcript_build = list()
    if id_reference in Se_assembly:
        seq_reference = Se_assembly[id_reference]
        if data.shape[0] > 1:
            for i in range(0, num_exons):
                exon_start = data[i][1]-1 
                exon_end = data[i][2]
                seq_exon = seq_reference[exon_start:exon_end].upper()
                seq_transcript_build.append(seq_exon)
                if i < (num_exons-1):
                    intron_start = exon_end
                    intron_end = data[i+1][1] - 1
                    seq_intron = seq_reference[intron_start:intron_end].lower()
                    seq_transcript_build.append(seq_intron)
        else:
            pos_start = data[0][1]-1
            pos_end = data[0][2]
            seq_exon = seq_reference[pos_start:pos_end]
            seq_transcript_build.append(seq_exon)
        seq_transcript = "".join(seq_transcript_build)
        if sense == "-":
            seq_transcript = reverse_complement(seq_transcript)
        d_id_premrna[id_transcript] = seq_transcript
    else:
        not_work.append(data)


In [6]:
%%time
# chromosomes = list()
# for file in os.scandir("./grch38/chromosomes/"):
#     if file.name.endswith(".gz"):
#         chromosomes.append(sy.read_fasta(file, verbose=True, description=False))
# Se_assembly = pd.concat(chromosomes)
Se_assembly = sy.io.read_fasta("/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm39/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz", description=False)


CPU times: user 30.4 s, sys: 1.92 s, total: 32.3 s
Wall time: 32.6 s


In [6]:
# import soothsayer as sy
# from soothsayer.utils import format_path, infer_compression
# def infer_compression(path:str):
#     path = format_path(path)
#     compression = None
#     ext_zip = (".zip")
#     ext_gzip = (".gz", ".gzip", ".pgz")
#     ext_bz2 = (".bz2", ".bzip2", ".pbz2")
#     if path.endswith(ext_gzip):
#         compression= "gzip"
#     if path.endswith(ext_bz2):
#         compression = "bz2"
#     if path.endswith(ext_zip):
#         compression = "zip"
#     return compression

# def get_file_object(path:str, mode="infer", compression="infer", safe_mode=False, verbose=True):
#     """
#     with get_file_object("./test.txt.zip", mode="infer", verbose=False) as f_read:
#     with get_file_object("./test_write.txt.bz2", mode="write", verbose=False) as f_write:
#         for line in f_read.readlines():
#             line = str(line.strip())
#             print(line, file=f_write)
#     """
#     # Init
#     f = None
#     write_file_object = False
    
#     # Paths
#     path = format_path(path)
#     path_exists = os.path.exists(path)

#     if verbose:
#         header = " ".join(["Processing file:", path])
#         print("="*len(header), header, "="*len(header), sep="\n", file=sys.stderr)
#     if compression == "infer":
#         compression = infer_compression(path)
#         if verbose:
#             print("Inferring compression:", compression, file=sys.stderr)

#     # Inferring mode
#     if mode == "infer": # Create new function for this? infer_filemode? 
#         if path_exists:
#             mode = "read"
#         else:
#             mode = "write"
#     assert mode != "infer", "The mode should be inferred by this point.  Please specify mode manually."
#     assert compression != "infer", "The compression should be inferred by this point.  Please specify compression manually."

#     # Generic read write
#     if mode in ["write", "read"]:
#         if mode == "write":
#             mode = "w"
#         if mode == "read":
#             mode = "r"
#         if compression in ["gzip", "bz2"]:
#             mode = mode + "b"
#         if verbose:
#             print("Inferring mode:", mode, file=sys.stderr)
            
#     # Will a file be written?
#     if "w" in mode:
#         write_file_object = True
        
#     # Ensure zip is not being written
#     if write_file_object:
#         assert compression != "zip", "Currently cannot handle writing zip files.  Please use gzip, bz2, or None."
#         # Future do this:
#         # https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.open
        
#     # Safe mode
#     if safe_mode:
#         if all([write_file_object, path_exists]):
#             raise Exception(f"Safe Mode: Please explicitly provide a writeable mode because `{path}` already exists and will be rewritten.")

#     # GZIP compression
#     if compression == "gzip":
#         f = gzip.open(path, mode)
#     # BZ2 compression
#     if compression == "bz2":
#         f = bz2.open(path, mode)
#     if compression == "zip":
#         filename, ext = os.path.splitext(os.path.split(path)[-1])
#         f = zipfile.ZipFile(path,mode).open(filename)
#     # No compression
#     if f is None:
#         return open(path, mode)
    
#     # Reading and writing compressed files
#     else:
#         return TextIOWrapper(f, encoding="utf-8")

    
# with get_file_object("/Users/jespinoz/test.txt.zip", mode="infer", verbose=False) as f_read:
#     with get_file_object("/Users/jespinoz/test_write.txt.bz2", mode="write", verbose=False) as f_write:
#         for line in f_read.readlines():
#             line = str(line.strip())
#             print(line, file=f_write)

In [149]:
# from soothsayer.utils import format_path, infer_compression#, get_file_object

# def read_gff3(path:str, compression="infer", record_type=None, verbose = True):
#     path = format_path(path)
#     if verbose: 
#         print(f"Reading gff3 file:",path,sep="\t", file=sys.stderr)
#     accepted_recordtypes = {"exon", "gene", "transcript", "protein", None}
#     assert record_type in accepted_recordtypes, f"Unrecognized record_type.  Please choose from the following: {accepted_recordtypes}"
    
#     # Read the gff3 file
#     with get_file_object(path, mode="read", compression=compression, safe_mode=False, verbose=False) as f:
#         if verbose:
#             iterable_lines = tqdm(f.readlines(), "Processing lines")
#         else:
#             iterable_lines = f.readlines()
#         data= list()
#         if record_type is None:
#             for line in iterable_lines:
#                 if not line.startswith("#"):
#                     line = line.strip("\n")
#                     base_fields = line.split("\t")
#                     data.append(base_fields)
#         else:
#             for line in iterable_lines:
#                 if not line.startswith("#"):
#                     if f"{record_type}_id" in line:
#                         line = line.strip("\n")
#                         base_fields = line.split("\t")
#                         data.append(base_fields)
                
#     # Generate table
#     df_base = pd.DataFrame(data)
#     df_base.columns = ["seq_record", "ensembl", "seq_type", "pos_start", "pos_end", ".1", "sense", ".2", "gff_data"]
#     # Splitting fields
#     if verbose:
#         iterable_fields = tqdm(enumerate(df_base.iloc[:,8].map(lambda x:dict([y.split("=") for y in x.split(";")])).as_matrix()), "Splitting fields")
#     else:
#         iterable_fields = enumerate(df_base.iloc[:,8].map(lambda x:dict([y.split("=") for y in x.split(";")])).as_matrix())
#     df_fields = pd.DataFrame(OrderedDict([(i,pd.Series(v)) for i,v in iterable_fields])).T
#     df_gff3 = pd.concat([df_base.iloc[:,:7], df_fields], axis=1)
    
#      # Contains identifier
#     if "ID" in df_gff3.columns:
#         df_gff3["seq_id"] = np.nan
#         mask_notnull = df_gff3["ID"].notnull()
#         df_gff3["seq_id"][mask_notnull] = df_gff3["ID"][mask_notnull].map(lambda id:id.split(":")[1])
#     return df_gff3

In [9]:
import re
gene_id "ENSMUSG00000102628"

SyntaxError: invalid syntax (<ipython-input-9-ac0b9cdf8458>, line 2)

In [7]:
# # Read Fasta File
# def read_fasta(path:str, description:bool=True, case="upper", func_header=None, into=pd.Series, compression="infer", name=None, verbose=True):
#     """
#     Reads in a single fasta file or a directory of fasta files into a dictionary.
#     """
#     # Get path
#     path = format_path(path)
    
#     # Assign pathname as name if there isn't one
#     if name is None:
#         name = path

#     # Open file object
#     f = get_file_object(path, mode="read", compression=compression, safe_mode=False, verbose=False)
    
#     # Read in fasta
#     d_id_seq = OrderedDict()

#     if verbose:
#         seq_records = tqdm(SeqIO.FastaIO.SimpleFastaParser(f), f"Reading sequence file: {path}")
#     else:
#         seq_records = SeqIO.FastaIO.SimpleFastaParser(f)

#     # Verbose but faster
#     if case == "lower":
#         for header, seq in seq_records:
#             seq = seq.lower()
#             if not description:
#                 header = header.split(" ")[0]
#             d_id_seq[header] = seq
#     if case == "upper":
#         for header, seq in seq_records:
#             seq = seq.upper()
#             if not description:
#                 header = header.split(" ")[0]
#             d_id_seq[header] = seq
#     if case is None:
#         for header, seq in seq_records:
#             if not description:
#                 header = header.split(" ")[0]
#             d_id_seq[header] = seq

#     # Close File
#     f.close()

#     # Transform header
#     if func_header is not None:
#         d_id_seq = OrderedDict( [(func_header(id),seq) for id, seq in d_id_seq.items()])
#     sequences = into(d_id_seq)
#     if hasattr(sequences, "name"):
#         sequences.name = name
#     return sequences

In [8]:
# for ext in ["",".gz", ".bz2", ".zip"]:
#     print(ext)
#     %timeit read_fasta(f"./cds.fa{ext}", verbose=False)
    

In [9]:
# df_gff3exons = read_gff3("/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz", record_type="exon")
# df_gff3genes = read_gff3("/Users/jespinoz/Homo_sapiens.GRCh37.87.chromosome.Y.gff3.gz", record_type="gene")

In [10]:
# ??infer_compression

In [27]:

# Forward direction is start-1:end
# str(Se_assembly["1 1 dna:chromosome chromosome:GRCh37:1:1:249250621:1 REF"][11869-1:12227])

# Reverse direction is reverse_complement(start-1:end)
# reverse_complement(str(Se_assembly["1 1 dna:chromosome chromosome:GRCh37:1:1:249250621:1 REF"][14363-1:14829]))

# Note for the reverse, the first exon that appears is the last in the sequence


'GCATCTCTGGGAAAGGACCTGGGGCTGGTGAGGGGCCCGGAGGAGCCTTTGCCCGCGTGTCAGACTCCATCCCTCCTCTGCCGCCACCGCAGCAGCCACAGGCAGAGGAGGACGAGGACGACTGGGAATCGTAGGGGGCTCCATGACACCTTCCCCCCCAGACCCAGACTTGGGCCGTTGCTCTGACATGGACACAGCCAGGACAAGCTGCTCAGACCTACTTCCTTGGGAGGGGGTGACGGAACCAGCACTGTGTGGAGACCAGCTTCAAGGAGCGGAAGGCTGGCTTGAGGCCACACAGCTGGGGCGGGGACTTCTGTCTGCCTGTGCTCCATGGGGGGACGGCTCCACCCAGCCTGCGCCACTGTGTTCTTAAGAGGCTTCCAGAGAAAACGGCACACCAATCAATAAAGAACTGAGCAGAAACCAACAGTGTGCTTTTAATAAAGGATCTCTAGCTGTGCAGG'

In [11]:
# df_gff

In [24]:
df_exons["transcript_id"]

/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm38.77/Mus_musculus.GRCm38.77.gtf
0        NaN
1        NaN
3        NaN
5        NaN
4        NaN
          ..
621802   NaN
621806   NaN
621803   NaN
621804   NaN
621805   NaN
Name: transcript_id, Length: 653441, dtype: float64

In [40]:
fields = ['seq_record', 'pos_start', 'pos_end', 'sense', 'transcript_id',  'exon_id']
df_data = df_exons.loc[:,fields].sort_values(["seq_record", "pos_start"], ascending=True)

for id_field in ['pos_start', 'pos_end']:
    df_exons[id_field] = df_data[id_field].astype(int)
d_id_premrna = OrderedDict()
not_work = list()
for id_transcript, _df_exon in tqdm(df_data.groupby("transcript_id", axis=0)):
#     id_transcript = id_transcript[11:]
#     print(id_transcript)
    data = _df_exon.values
    num_exons = data.shape[0]
    id_reference = str(data[0][0])
    sense = data[0][3]
    seq_transcript_build = list()
    if id_reference in Se_assembly:
        seq_reference = Se_assembly[id_reference]
        if data.shape[0] > 1:
            for i in range(0, num_exons):
                exon_start = data[i][1]-1 
                exon_end = data[i][2]
                seq_exon = seq_reference[exon_start:exon_end].upper()
                seq_transcript_build.append(seq_exon)
                if i < (num_exons-1):
                    intron_start = exon_end
                    intron_end = data[i+1][1] - 1
                    seq_intron = seq_reference[intron_start:intron_end].lower()
                    seq_transcript_build.append(seq_intron)
        else:
            pos_start = data[0][1]-1
            pos_end = data[0][2]
            seq_exon = seq_reference[pos_start:pos_end]
            seq_transcript_build.append(seq_exon)
        seq_transcript = "".join(seq_transcript_build)
        if sense == "-":
            seq_transcript = reverse_complement(seq_transcript)
        d_id_premrna[id_transcript] = seq_transcript
    else:
        not_work.append(data)


100%|██████████| 100249/100249 [00:53<00:00, 1866.22it/s]


In [42]:
Se_premrna = pd.Series(d_id_premrna)
# Se_assembly.index
Se_premrna.size

99934

In [7]:
# Not ready
from soothsayer.utils import is_dict_like
# Need to group proteins, genes, and transcripts together. have exon separate. have intron w/ unique identifiers based on exons. have premrna a custom one.
def get_regions_from_gff3(path_gff3:str, path_sequences:str, record_type="premrna"):
    
    def _get_exons():
        
    def _get_introns():
    
    def _get_premrna():
        
    def _get_transcripts():
        
    def _get_genes():
        
        
    # Get gff3
    if is_type(path_gff3, pd.DataFrame):
        df_gff3 = path_gff3
    else:
        df_gff3 = read_gff3(path, compression=compression, record_type="exon", verbose=False)
    idx_exons = df_gff3["exon_id"].dropna().index
    df_gff3 = df_gff3.loc[idx_exons,:]
    
    # Get sequences
    if is_dict_like(path_sequences):
        Se_sequences = path_sequences
    else:
        Se_sequences = read_fasta(path_sequences, description=False, verbose=False)

    d_id_seq = OrderedDict()
    for id_transcript, df_gff3__exon in tqdm(df_gff3.groupby("Parent", axis=0)):
        id_transcript = id_transcript[11:]
        data = df_gff3__exon.values
        num_exons = data.shape[0]
        id_reference = str(data[0][0])
        sense = data[0][3]
        seq_reference = Se_sequences[id_reference]
        seq_transcript_build = list()
        if data.shape[0] > 1:
            for i in range(0, num_exons):
                exon_start = data[i][1]-1 
                exon_end = data[i][2]
                seq_exon = seq_reference[exon_start:exon_end].upper()
                seq_transcript_build.append(seq_exon)
                if i < (num_exons-1):
                    intron_start = exon_end
                    intron_end = data[i+1][1] - 1
                    seq_intron = seq_reference[intron_start:intron_end].lower()
                    seq_transcript_build.append(seq_intron)
        else:
            pos_start = data[0][1]-1
            pos_end = data[0][2]
            seq_exon = seq_reference[pos_start:pos_end]
            seq_transcript_build.append(seq_exon)
        seq_transcript = "".join(seq_transcript_build)
        if sense == "-":
            seq_transcript = reverse_complement(seq_transcript)
        d_id_seq[id_transcript] = seq_transcript

In [46]:
# sy.write_fasta(Se_premrna, "./grch38/Homo_sapiens.GRCh38.94.premrna.fa")
# sy.io.write_fasta(Se_premrna, "/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm38.77/Mus_musculus.GRCm38.77.premrna.fa")
Se_premrna["ENSMUST00000000001"]

'...'

In [48]:
Se_cds = sy.io.read_fasta("/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm38.77/Mus_musculus.GRCm38.cds.all.fa", description=False, func_header=lambda id_transcript:id_transcript.split(".")[0])


Reading sequence file: /Users/jespinoz/Documents/NMD/Mus_musculus.GRCm38.77/Mus_musculus.GRCm38.cds.all.fa: 52998it [00:01, 51239.33it/s]


'ATGGGCTGCACGTTGAGCGCCGAGGACAAGGCGGCGGTGGAGCGGAGCAAGATGATCGACCGCAACTTGCGGGAGGACGGGGAGAAAGCGGCCAAAGAAGTGAAGCTGCTGCTGCTCGGCGCTGGAGAATCTGGTAAAAGTACCATCGTGAAACAGATGAAAATCATTCATGAGGACGGCTATTCAGAGGACGAATGTAAACAGTATAAAGTAGTTGTCTACAGCAATACTATTCAGTCCATCATTGCAATCATACGAGCCATGGGACGGTTGAAGATTGATTTTGGGGAATCTGCCAGAGCAGATGATGCCCGACAGTTATTTGTTTTAGCTGGGAGTGCTGAAGAAGGAGTCATGACTTCAGAACTAGCAGGCGTGATTAAACGTTTATGGCGAGATGGCGGGGTACAGGCATGCTTTAGCAGGTCCAGGGAATATCAGCTCAATGATTCTGCTTCATACTACCTAAATGATTTGGATAGAATATCCCAGACCAACTACATTCCAACTCAGCAAGATGTTCTTCGGACAAGAGTGAAGACTACAGGCATTGTGGAGACCCACTTCACCTTCAAGGAACTCTACTTCAAAATGTTTGATGTAGGTGGCCAAAGATCCGAACGAAAAAAGTGGATTCACTGTTTTGAGGGAGTGACAGCAATTATCTTTTGTGTGGCTCTCAGTGATTACGACCTTGTTCTGGCTGAGGATGAGGAAATGAACCGAATGCATGAAAGCATGAAATTGTTTGACAGCATTTGTAACAACAAATGGTTTACAGACACTTCAATCATTCTCTTTCTTAATAAGAAAGACCTTTTTGAGGAAAAAATAAAGAGGAGTCCATTAACAATCTGTTATCCAGAATACACAGGTTCCAATACATACGAAGAGGCAGCTGCTTACATTCAGTGCCAGTTTGAAGATCTGAACCGAAGAAAAGATACCAAGGAGGTCTACACTCACTTCACCTGTGCCACAGACACCAAAAATGTGCAG

In [49]:
sy.io.write_fasta(Se_cds, "/Users/jespinoz/Documents/NMD/Mus_musculus.GRCm38.77/Mus_musculus.GRCm38.cds.all.no-description.fa")

In [19]:
# Se_premrna.map(len).compress(lambda x:x > 10000).sort_values().head()

ENST00000441558    10001
ENST00000513749    10001
ENST00000548721    10001
ENST00000523248    10001
ENST00000575320    10001
dtype: int64

In [20]:
# Se_premrna["ENST00000441558"]

'GGTCCACCCGCGAATCCTCTACAGAGATGGCCAAGCGCTGGGGTCGTGGCTGCGTCCTCTAGGCCAGgtgagaatttacaacccaccctcggccactgttgtgctgtggccgctgatgattaagcatgaggaactcagaaactctgaaccttctgacaaacgggttctcccctgcccaccgcggtgccccaaacccagcaggacagattggtttacatggtggggcattggggaggacccagagctttcaataagggaagaaatgatgtggggagggacagtaaatcaaagtaggggattaggccgggcgctgtggctcacgtctgtaatcccagccctttgggaggctgaggcagatggatctctcgagactgggagttctagaccagcctggccaacatggcgaaaccccatctctactaaaaatacgaaaattagccgggcgtggtggcgcgcgcctgtaattccagctactagagaggctgaggcaggagaatcgcttgaaccctggaggcagaggttgcaatgaggcaagatggcgccactgcactccagcctgggctacagagcgagactgtctcaaaaaaaaaaaaaaaaaaaagtaggggattagagctggccacagcttccccagtaacctgcagttactggagcaggaaggggcctcggagaatccctggaagcatctcaacagaatccagaggggacaagaaaagaggagggatacgtctgtcgcatccttcccttctcattgatccaagtttgcactagtggtgtgagccctccttatcttccaggccttggtgggcagctgttgacatgtccagaaccccaggaagtgctgaaggtacctgcagttcaggccccatggaagcccagacctccttcacccttcggggacactgaaattggccaaatgtttaggccatgaggatggctgctgagtaagcccataatccctacatcggacacaagtaagcaggtgaaggcctacaggta

In [1]:
import soothsayer as sy

In [ ]:
sy.microbiome.extract_intergenic_sequences()